In [ ]:
import geopandas as gpd
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
from matplotlib.patches import FancyArrow
from matplotlib.patches import FancyArrowPatch
import xlrd
import folium
from folium.plugins import FloatImage
from branca.colormap import LinearColormap

In [ ]:
# Set path
os.chdir('/Users/sunny/Library/CloudStorage/OneDrive-Personal/Documents/Python/Electoral maps project')

In [ ]:
pwd

**NDA 2024 vs 2019**

In [ ]:
# Load geodatasets
geo_nda_2024 = gpd.read_file("geo_nda_2024.geojson")

In [ ]:
geo_nda_2019 = gpd.read_file("geo_nda_2019.geojson")

In [ ]:
# Change these column names in the main analysis script
geo_nda_2019.rename(columns={"Party Name" : "Party",
                              "Votes" : "Total Votes", 
                              "Total votes" : "Total Votes Cast", 
                              "Vote share" : "Vote Share (%)"}, inplace=True)

In [ ]:
# label cols as 2019 and 2024
geo_nda_2019.columns
cols_to_tag = ['Candidate', 'Party', 'Total Votes', 'Total Votes Cast', 'Vote Share (%)']

# Rename columns in the 2019 dataset
geo_nda_2019 = geo_nda_2019.rename(columns={col: f"{col} (2019)" for col in cols_to_tag})

# Rename columns in the 2024 dataset
geo_nda_2024 = geo_nda_2024.rename(columns={col: f"{col} (2024)" for col in cols_to_tag})

In [ ]:
geo_nda_2019.head()

In [ ]:
geo_nda_2024.head()

In [ ]:
print(geo_nda_2019.columns)
print(geo_nda_2024.columns)

In [ ]:
cols_to_drop_2019 = ['Total Votes (2019)', 'Total Votes Cast (2019)', 'nda_tag', 'Reserved status', 'Reservation status']
cols_to_drop_2024 = ['Total Votes (2024)', 'Total Votes Cast (2024)', 'nda_tag', 'Reserved status']

In [ ]:
geo_nda_2019 = geo_nda_2019.drop(columns=[col for col in cols_to_drop_2019 if col in geo_nda_2019.columns])

In [ ]:
geo_nda_2024 = geo_nda_2024.drop(columns=[col for col in cols_to_drop_2024 if col in geo_nda_2024.columns])

In [ ]:
geo_nda_compare = pd.merge(geo_nda_2019, geo_nda_2024, on=["State", "Constituency", "geometry"], how="outer")

In [ ]:
geo_nda_compare[(geo_nda_compare["Candidate (2019)"].isna()) | (geo_nda_compare["Candidate (2024)"].isna())]
# 20 out of 543 - pretty good
# Issues: 8 in Assam;
# 6 in J&K/Ladakh;
# 4 (2*2) Daman / Dadra state changed;
# 2 (1*2) Hardwar 2019 spelling

In [ ]:
geo_nda_compare["Vote Swing"] = geo_nda_compare["Vote Share (%) (2024)"] - geo_nda_compare["Vote Share (%) (2019)"]

In [ ]:
geo_nda_compare[geo_nda_compare["Vote Swing"].isna()]

In [ ]:
# Plot

In [ ]:
# Load the shapefile for India's boundary (this should only have the boundary of India, not the districts)
india_boundary = gpd.read_file('maps-master/India-State-and-Country-Shapefile-Updated-Jan-2020-master/India_Country_Boundary.shp')

In [ ]:
india_boundary

In [ ]:
# Plot the India map, check that it makes sense
fig, ax = plt.subplots(figsize=(15, 15))
india_boundary.boundary.plot(ax=ax, linewidth=0.15)
plt.show()

In [ ]:
# Basic map, NDA
# No NDA candidates - Kashmir Valley; plus Assam will be dealt with later
nda_candidates = geo_nda_compare[geo_nda_compare["Vote Swing"].notna()]
no_nda_candidates = geo_nda_compare[geo_nda_compare["Vote Swing"].isna()]

# Define a diverging colormap (blue for negative, white for zero, orange for positive)
cmap = mcolors.LinearSegmentedColormap.from_list('diverging_cmap', ['#0000FF', '#FFFFFF', '#FF6600'])

# Plot the GeoDataFrame
fig, ax = plt.subplots(1, 1, figsize=(8, 8))
nda_candidates.plot(column="Vote Swing", 
                    cmap=cmap, 
                    linewidth=0.1, 
                    edgecolor="gray", 
                    ax=ax, 
                    legend=True, 
                    vmin = -100, 
                    vmax = 100)

no_nda_candidates.plot(color="#D3D3D3", linewidth=0.05, edgecolor="gray", ax=ax)

# Add a title
ax.set_title("NDA 2024 vs 2019 vote swing", fontsize=16)

# Remove axis for better visualization
ax.axis('off')

# Show the plot
plt.show()

***Interactive map***

In [ ]:
# Create a colormap
colors = ['blue', 'white', 'orange']
colormap = LinearColormap(
    colors=colors,
    vmin=-100,
    vmax=100
).to_step(10)
colormap.caption = "NDA Vote Swing: 2024 vs 2019"

# Initialize the map
m = folium.Map(location=[20.5937, 78.9629], zoom_start=10, tiles=None)

# Add GeoJSON data
geojson = folium.GeoJson(
    geo_nda_compare.to_json(),
    style_function=lambda feature: {
        'fillColor': (
            '#D3D3D3' if feature['properties']['Vote Swing'] in [None, ''] 
            else colormap(float(feature['properties']['Vote Swing']))
        ),
        'color': 'black',
        'weight': 0.5,
        'fillOpacity': 0.7,
    },
    tooltip=folium.GeoJsonTooltip(
        fields=['State', 'Constituency', 'Vote Swing'],
        aliases=['State:', 'Constituency:', 'Vote Swing:'],
        localize=True
    )
).add_to(m)

# Fit the map to the bounds of the GeoDataFrame
bounds = geo_nda_compare.total_bounds  # [minx, miny, maxx, maxy]
m.fit_bounds([[bounds[1], bounds[0]], [bounds[3], bounds[2]]])

# Add the colormap to the map
colormap.add_to(m)

# Inject CSS for a white background
from folium.plugins import FloatImage

css = """
<style>
    .leaflet-container {
        background: #FFFFFF !important;
    }
</style>
"""
folium.Html(css, script=True).add_to(m)

# Save and display the map
m.save("nda_vote_swing_map.html")
m

***Plot with arrows***

In [ ]:
print(geo_nda_compare.crs)

In [ ]:
print(india_boundary.crs)

In [ ]:
geo_nda_compare = geo_nda_compare.to_crs(epsg=7755)

In [ ]:
india_boundary = india_boundary.to_crs(epsg=7755)

In [ ]:
# Calculate constituency centroids
geo_nda_compare['centroid'] = geo_nda_compare['geometry'].centroid
geo_nda_compare['centroid_x'] = geo_nda_compare['centroid'].x
geo_nda_compare['centroid_y'] = geo_nda_compare['centroid'].y

In [ ]:
# Plot
fig, ax = plt.subplots(figsize=(12, 12))
india_boundary.plot(ax=ax, color='none', edgecolor='black', linewidth=0.25, zorder=1)

# Get the maximum vote swing for scaling
max_swing = geo_nda_compare['Vote Swing'].abs().max()

# Add arrows
for _, row in geo_nda_compare.iterrows():
    x_start, y_start = row['centroid_x'], row['centroid_y']
    x_end = x_start + (row['Vote Swing'] / max_swing) * 300000  # Adjust scale factor
    y_end = y_start + (abs(row['Vote Swing']) / max_swing) * 300000  # Adjust scale factor
    # Create the arrow with a tip
    arrow = FancyArrowPatch(
        (x_start, y_start), (x_end, y_end),  # Start and end points
        mutation_scale=8,                   # Size of the arrow head (tip)
        color='blue' if row['Vote Swing'] < 0 else 'orange',  # Arrow color
        arrowstyle='-|>',                    # Use an arrowhead style
        linewidth=0.5,
        zorder=2  # Ensure the arrows are above the boundaries
    )
    ax.add_patch(arrow)

# Set title and axis options
ax.set_title('Election Swing Visualization', fontsize=16)
plt.axis('off')
plt.show()

***Interactive map with swings as arrows***

In [ ]:
# Data cleaning steps

In [ ]:
# Ensure the CRS is correct (in latitude/longitude coordinates, EPSG:4326)
print(geo_nda_compare.crs)
geo_nda_compare = geo_nda_compare.to_crs(epsg=4326)

In [ ]:
print(india_boundary.crs)
india_boundary = india_boundary.to_crs(epsg=4326)

In [ ]:
# Remove any rows where vote swing is missing
geo_nda_compare = geo_nda_compare.dropna(subset=['Vote Swing'])

In [ ]:
# Calculate constituency centroids
# Step 1: Reproject the GeoDataFrame to a projected CRS
geo_projected = geo_nda_compare.to_crs(epsg=3857)  # Web Mercator or another suitable projected CRS

# Step 2: Calculate centroids in the projected CRS
geo_projected['centroid'] = geo_projected['geometry'].centroid

# Step 3: Reproject centroids back to geographic CRS (EPSG:4326)
geo_nda_compare = geo_nda_compare.to_crs(epsg=4326)
geo_nda_compare['centroid'] = geo_projected['centroid'].to_crs(epsg=4326)

# Now, geo_nda_compare['centroid'] contains valid lat/lon values

In [ ]:
# Create arrow head
from math import atan2, degrees

# Function to create a custom SVG for the arrowhead
def create_arrowhead_icon(color, angle):
    arrowhead_svg = f"""
    <svg xmlns="http://www.w3.org/2000/svg" width="24" height="24" viewBox="0 0 24 24">
        <polygon points="12,0 0,24 24,24" 
                 fill="{color}" 
                 transform="rotate({angle}, 12, 12)"/>
    </svg>
    """
    return folium.CustomIcon(icon_image=arrowhead_svg, icon_size=(24, 24))


In [ ]:
# Create a colormap for tooltips
colors = ['blue', 'white', 'orange']
colormap = LinearColormap(
    colors=colors,
    vmin=-100,
    vmax=100
).to_step(10)
colormap.caption = "NDA Vote Swing: 2024 vs 2019"

# Initialize the map centered on India
m = folium.Map(location=[20.5937, 78.9629], zoom_start=5, tiles=None)

# Plot the boundary of India
folium.GeoJson(
    india_boundary.to_json(),  # Get the boundary's geojson interface
    style_function=lambda feature: {
        'color': 'black',  # Color of the boundary line
        'weight': 1,       # Line thickness
        'fillOpacity': 0,  # No fill for the boundary
    }
).add_to(m)

# Add arrows for each district based on the vote swing
max_swing = geo_nda_compare['Vote Swing'].abs().max()  # Get max swing to scale the arrows

for _, row in geo_nda_compare.iterrows():
    # Get centroid coordinates for the arrow start point
    x_start, y_start = row['centroid'].x, row['centroid'].y
    
    # Calculate the arrow's end coordinates based on the vote swing
    # Scale the swing to control the size of the arrows
    scale_factor = 1.5  # You can adjust this value to make the arrows bigger or smaller
    x_end = x_start + (row['Vote Swing'] / max_swing) * scale_factor
    y_end = y_start + (abs(row['Vote Swing']) / max_swing) * scale_factor
    
    # Arrow color based on the swing (blue for away from NDA, orange for towards NDA)
    arrow_color = 'blue' if row['Vote Swing'] < 0 else 'orange'

    # Create a polyline (line) to represent the arrow
    arrow_line = folium.PolyLine(
        locations=[[y_start, x_start], [y_end, x_end]],  # Using [lat, lon] for coordinates
        color=arrow_color,
        weight=2,
        opacity=0.7
    )
    
    # Add a tooltip to the arrow
    tooltip_text = f"Constituency: {row['Constituency']}<br>Vote Swing: {row['Vote Swing']:.2f}"
    tooltip = folium.Tooltip(tooltip_text)
    arrow_line.add_child(tooltip)

    # Add the arrow to the map
    arrow_line.add_to(m)

    # Calculate the angle of rotation for the arrowhead
    angle = degrees(atan2(y_end - y_start, x_end - x_start))
    
    # Add the arrowhead as a custom SVG marker
    folium.Marker(
        location=[y_end, x_end],  # The arrow tip coordinates
        icon=create_arrowhead_icon(color=arrow_color, angle=angle)
    ).add_to(m)

# Fit the map to the bounds of the GeoDataFrame
bounds = geo_nda_compare.total_bounds  # [minx, miny, maxx, maxy]
m.fit_bounds([[bounds[1], bounds[0]], [bounds[3], bounds[2]]])

# Add the colormap for the tooltips
colormap.add_to(m)

# Inject CSS for a white background
from folium.plugins import FloatImage

css = """
<style>
    .leaflet-container {
        background: #FFFFFF !important;
    }
</style>
"""
folium.Html(css, script=True).add_to(m)

# Save the map as HTML
m.save("nda_vote_swing_arrows_map.html")
m

In [ ]:
# Ensure the CRS is correct (in latitude/longitude coordinates, EPSG:4326)
geo_nda_compare = geo_nda_compare.to_crs(epsg=4326)
india_boundary = india_boundary.to_crs(epsg=4326)

# Remove any rows where centroid or vote swing is missing
geo_nda_compare = geo_nda_compare.dropna(subset=['Vote Swing'])

# Create a colormap for tooltips
colors = ['blue', 'white', 'orange']
colormap = LinearColormap(
    colors=colors,
    vmin=-100,
    vmax=100
).to_step(10)
colormap.caption = "NDA Vote Swing: 2024 vs 2019"

# Initialize the map centered on India
m = folium.Map(location=[20.5937, 78.9629], zoom_start=10, tiles=None)

# Plot the boundary of India
folium.GeoJson(
    india_boundary.to_json(),  # Get the boundary's geojson interface
    style_function=lambda feature: {
        'color': 'black',  # Color of the boundary line
        'weight': 1,       # Line thickness
        'fillOpacity': 0,  # No fill for the boundary
    }
).add_to(m)

# Fit the map to the bounds of the GeoDataFrame
bounds = india_boundary.total_bounds  # [minx, miny, maxx, maxy]
m.fit_bounds([[bounds[1], bounds[0]], [bounds[3], bounds[2]]])

# Add the colormap for the tooltips
colormap.add_to(m)

# Inject CSS for a white background
from folium.plugins import FloatImage

css = """
<style>
    .leaflet-container {
        background: #FFFFFF !important;
    }
</style>
"""
folium.Html(css, script=True).add_to(m)

# Save the map as HTML
m.save("nda_vote_swing_arrows_map.html")
m